In [6]:
import os
import torch
from tqdm.auto import tqdm
from diffusion_3d.chestct.diffusion.ldm.model import LDM3DLightning
from diffusion_3d.chestct.diffusion.ldm.config import get_config
from diffusion_3d.datasets.ct_rate import CTRATEDataset
from arjcode.visualize import plot_scans

In [7]:
config = get_config()
config

Munch({'data': Munch({'csvpath': '/raid3/arjun/ct_pretraining/csvs/sources.csv', 'datapath': '/raid3/arjun/ct_pretraining/scans/', 'checkpointspath': '/raid3/arjun/checkpoints/diffusion_3d/', 'limited_dataset_size': None, 'allowed_spacings': ((0.4, 7), (-1, -1), (-1, -1)), 'allowed_shapes': ((96, -1), (256, -1), (256, -1)), 'train_augmentations': Munch({'_target_': 'monai.transforms.Compose', 'transforms': [Munch({'_target_': 'vision_architectures.transforms.croppad.CropForegroundWithCropTrackingd', 'keys': ['image'], 'source_key': 'image', 'allow_smaller': True}), Munch({'_target_': 'monai.transforms.ScaleIntensityRanged', 'keys': ['image'], 'a_min': -1000, 'a_max': 1000, 'b_min': 0.0, 'b_max': 1.0, 'clip': True}), Munch({'_target_': 'vision_architectures.transforms.croppad.RandSpatialCropSamplesWithCropTrackingd', 'keys': ['image'], 'roi_size': (76, 76, 76), 'max_roi_size': (115, 115, 115), 'random_size': True, 'num_samples': 10}), Munch({'_target_': 'vision_architectures.transforms.

In [8]:
checkpoint_path = r"/raid3/arjun/checkpoints/diffusion_3d/v3__2025_05_06/version_0/checkpoints/last.ckpt"
os.path.exists(checkpoint_path)

False

In [9]:
device = torch.device("cpu")
# device = torch.device("cuda:1")

In [ ]:
model = LDM3DLightning.load_from_checkpoint(checkpoint_path, map_location="cpu")
model = model.to(device)
model.eval()

/home/users/arjun.agarwal/miniforge3/envs/env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/users/arjun.agarwal/miniforge3/envs/env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Diffusion3DLightning(
  (denoiser): Diffusion3D(
    (encoder_mapping): CNNBlock3D(
      (conv): Conv3d(1, 18, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same)
      (checkpointing_level1): ActivationCheckpointing(enabled=False)
    )
    (encoder): Encoder(
      (stages): ModuleList(
        (0): ModuleList(
          (0): StageBlock(
            (position_embeddings): AbsolutePositionEmbeddings3D()
            (layers): ModuleList(
              (0-3): 4 x ResBlock(
                (conv1): CNNBlock3D(
                  (conv): Conv3d(18, 18, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same)
                  (norm): GroupNorm(6, 18, eps=1e-05, affine=True)
                  (act): SiLU()
                  (checkpointing_level1): ActivationCheckpointing(enabled=False)
                )
                (conv2): CNNBlock3D(
                  (conv): Conv3d(18, 18, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same)
                  (norm): GroupNorm(6, 18, eps=1e-05, 

# Check denoising

In [10]:
dataset = CTRATEDataset(config.data, "test")

test:   0%|          | 0/500 [00:00<?, ?it/s]

No. of test datapoints: 500


In [11]:
batch = dataset[0]

x0 = batch["image"][None].to(device)
spacing = torch.tensor(batch["Spacing"])[None].to(device)

x0.shape, spacing.shape

(torch.Size([1, 1, 96, 96, 96]), torch.Size([1, 3]))

In [43]:
T = 750
noise = torch.randn_like(x0)
t = torch.tensor((T,), device=device)
guidance = torch.ones((2, 1), device=device)

In [44]:
xT = model.noise_scheduler.add_noise(x0, t, noise)

print(xT.min(), xT.max())
plot_scans([x0[0, 0, 40:41].cpu(), xT[0, 0, 40:41].cpu()], ["x0", f"x{T}"])

metatensor(-4.3632, device='cuda:1') metatensor(5.3883, device='cuda:1')


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

In [45]:
with torch.no_grad():
    xt = xT

    for timestep in tqdm([*range(T - 1, 0, -1)]):
        t = torch.tensor((timestep,), device=device)
        noise_pred = model(xt, t, spacing, guidance)

        x0_hat, xt_minus_1 = model.noise_scheduler.remove_noise(xt, t, noise_pred, eta=1.0)

        if (timestep + 1) % 50 == 0 or t == 1:
            print(f"timestep: {timestep}")
            print(f"xt: {xt.min()}, {xt.max()}")
            print(f"noise: {noise.min()}, {noise.max()}")
            print(f"noise_pred: {noise_pred.min()}, {noise_pred.max()}")
            print(f"x0_hat: {x0_hat.min()}, {x0_hat.max()}")
            print(f"xt_minus_1: {xt_minus_1.min()}, {xt_minus_1.max()}")
            print(f'L2: {torch.nn.functional.mse_loss(noise_pred, noise)}')
            plot_scans(
                [
                    noise_pred[0, 0, 40:41].cpu().numpy(),
                    x0[0, 0, 40:41].detach().cpu().numpy(),
                    x0_hat[0, 0, 40:41].detach().cpu().numpy(),
                    xt_minus_1[0, 0, 40:41].detach().cpu().numpy(),
                    xT[0, 0, 40:41].detach().cpu().numpy(),
                ],
                ["noise_pred", "x0", "x0_hat", "xt_minus_1", f"x{T}"],
            )

        xt = xt_minus_1

  0%|          | 0/749 [00:00<?, ?it/s]

timestep: 749
xt: -4.363170146942139, 5.388294219970703
noise: -4.759611129760742, 5.570115566253662
noise_pred: -4.792418479919434, 6.78438138961792
x0_hat: -2.298257350921631, 1.2292275428771973
xt_minus_1: -4.367315292358398, 5.0
L2: 0.001097226864658296


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

timestep: 699
xt: -4.040408134460449, 4.336990833282471
noise: -4.759611129760742, 5.570115566253662
noise_pred: -4.912566184997559, 5.373050689697266
x0_hat: -1.2522646188735962, 1.2348976135253906
xt_minus_1: -4.1498122215271, 4.339720726013184
L2: 0.37131473422050476


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

...

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

timestep: 49
xt: -0.9266924262046814, 1.3226107358932495
noise: -4.759611129760742, 5.570115566253662
noise_pred: -4.752572536468506, 4.599980354309082
x0_hat: -0.8125314712524414, 1.0277082920074463
xt_minus_1: -0.9397954940795898, 1.3226592540740967
L2: 1.886609673500061


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

timestep: 1
xt: -0.8152258992195129, 1.0205870866775513
noise: -4.759611129760742, 5.570115566253662
noise_pred: -0.4433196783065796, 1.1588640213012695
x0_hat: -0.8152258992195129, 1.0205870866775513
xt_minus_1: -0.8152258992195129, 1.0205870866775513
L2: 1.0030632019042969


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

# Generate samples

In [36]:
from vision_architectures.schedulers.noise import CosineNoiseScheduler

In [46]:
num_samples = 1

spacings = torch.tensor([1.0, 0.5, 0.5], device=device)
guidance = torch.ones((2, num_samples), device=device)

In [47]:
xt = torch.randn(num_samples, config.model.in_channels, *config.input_size, device=device)
xt.shape

torch.Size([1, 1, 96, 96, 96])

In [48]:
T = model.noise_scheduler.T
ddim_t_offset = T // 50
ddim_t_offset = 1

with torch.no_grad():
    for timestep in tqdm(range(T, 0, -ddim_t_offset)):
        t = torch.full((num_samples,), timestep, device=device)
        noise_pred = model(xt, t, spacings, guidance)

        x0_hat, xt_minus_1 = model.noise_scheduler.remove_noise(xt, t, noise_pred, prev_t_offset=ddim_t_offset, eta=1.)

        if timestep % 50 == 0 or t == ddim_t_offset:
            print(f"timestep: {timestep}")
            print(f"xt: {xt.min()}, {xt.max()}")
            print(f"noise_pred: {noise_pred.min()}, {noise_pred.max()} | {noise_pred.mean()}, {noise_pred.std()}")
            print(f"x0_hat: {x0_hat.min()}, {x0_hat.max()}")
            print(f"xt_minus_1: {xt_minus_1.min()}, {xt_minus_1.max()}")
            plot_scans(
                [
                    noise_pred[0, 0, 40:41].cpu().numpy(),
                    x0_hat[0, 0, 40:41].detach().cpu().numpy(),
                    xt_minus_1[0, 0, 40:41].detach().cpu().numpy(),
                ],
                ["noise_pred", "x0_hat", f"xt_minus_{ddim_t_offset}"],
            )

        xt = xt_minus_1

  0%|          | 0/1000 [00:00<?, ?it/s]

timestep: 1000
xt: -5.207174777984619, 4.861217498779297
noise_pred: -5.242274761199951, 5.333522796630859 | 0.0021260040812194347, 0.9978124499320984
x0_hat: -5.0, 5.0
xt_minus_1: -4.888940334320068, 4.572839260101318


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

timestep: 950
xt: -5.0, 5.0
noise_pred: -4.916719913482666, 6.680548191070557 | 0.001684906892478466, 1.0021768808364868
x0_hat: -5.0, 3.537106513977051
xt_minus_1: -4.857367515563965, 5.0


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

...

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

timestep: 50
xt: -2.2994866371154785, 1.408508539199829
noise_pred: -4.12729024887085, 4.484928131103516 | -0.007741330657154322, 0.8706355094909668
x0_hat: -2.163160800933838, 1.3187366724014282
xt_minus_1: -2.3035640716552734, 1.3947064876556396


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

timestep: 1
xt: -2.211871385574341, 1.2753937244415283
noise_pred: -1.5889955759048462, 1.1262389421463013 | -0.033661700785160065, 0.18516898155212402
x0_hat: -2.211871385574341, 1.2753937244415283
xt_minus_1: -2.211871385574341, 1.2753937244415283


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…